<a href="https://colab.research.google.com/github/skywalker0803r/works/blob/master/%E8%BC%95%E6%B2%B9%E6%A1%88%E5%AD%90%E9%9B%86%E6%88%90%E5%AD%B8%E7%BF%92%E5%AF%A6%E9%A9%97(%E5%85%A8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [146]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import ART2_Ricky
import pandas as pd
import numpy as np

# 定義參數

In [0]:
import numpy as np

#level 1 models
svm_model_dir_path = '/content/drive/My Drive/台塑輕油案子/集成學習/svm_model/'
rf_model_dir_path = '/content/drive/My Drive/台塑輕油案子/集成學習/rf_model/'
knn_model_dir_path = '/content/drive/My Drive/台塑輕油案子/集成學習/knn_model/'

#level 2 model
xgb_model_dir_path = '/content/drive/My Drive/台塑輕油案子/集成學習/xgboost_model/'

測試資料路徑 = '/content/drive/My Drive/台塑輕油案子/test_without_outlier.csv'

# 載入訓練資料

In [149]:
df = pd.read_csv(訓練資料路徑)
x_columns = ['T10','T50','T90','N+A in Feed']
y_columns = ['C5NP', 'C5IP', 'C5N', 'C6NP',
       'C6IP', 'C6N', 'C6A', 'C7NP', 'C7IP', 'C7N', 'C7A', 'C8NP', 'C8IP',
       'C8N', 'C8A', 'C9NP', 'C9IP', 'C9N', 'C9A', 'C10NP', 'C10IP', 'C10N',
       'C10A']
df = df[x_columns+y_columns]
df = df.rename(columns={'N+A in Feed':'NA'})
y_columns = df.columns[4:27].tolist()
x_columns = df.drop(y_columns,axis=1).columns.tolist()
print('x_columns:',x_columns)
print('y_columns:',y_columns)
print(df.shape)
df.head()

x_columns: ['T10', 'T50', 'T90', 'NA']
y_columns: ['C5NP', 'C5IP', 'C5N', 'C6NP', 'C6IP', 'C6N', 'C6A', 'C7NP', 'C7IP', 'C7N', 'C7A', 'C8NP', 'C8IP', 'C8N', 'C8A', 'C9NP', 'C9IP', 'C9N', 'C9A', 'C10NP', 'C10IP', 'C10N', 'C10A']
(4521, 27)


,T10,T50,T90,NA,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,100.5,119.2,146.5,31.978,0.272,0.132,0.059,5.293,2.570,2.819,0.494,10.395,8.070,6.411,2.917,9.138,9.649,4.810,5.373,6.405,9.759,4.590,3.661,0.875,5.257,0.525,0.319
1,99.6,117.9,145.5,31.568,0.297,0.151,0.062,5.089,2.531,2.804,0.499,10.074,7.958,6.395,2.894,8.970,9.548,4.753,5.443,6.324,9.899,4.301,2.995,0.881,5.591,1.119,0.303
2,100.0,118.8,145.6,31.344,0.262,0.126,0.061,5.107,2.571,2.771,0.492,10.069,7.913,6.378,2.890,9.006,9.591,4.778,5.468,6.360,9.983,4.274,2.979,0.865,5.641,0.964,0.289
3,100.4,118.6,142.9,31.453,0.224,0.105,0.060,4.894,2.497,2.650,0.469,10.015,7.685,6.376,2.866,9.133,9.708,4.889,5.510,6.444,10.182,4.420,2.964,0.830,5.637,0.968,0.281
4,100.4,118.1,142.2,32.190,0.243,0.117,0.061,4.946,2.503,2.695,0.477,10.053,7.765,6.394,2.877,9.101,9.676,4.855,5.500,6.416,10.115,4.347,3.725,0.835,4.823,0.969,0.290


# 切割訓練/驗證資料

In [150]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split( df ,test_size=0.1,random_state=42)
train_df.shape,val_df.shape

((4068, 27), (453, 27))

# 載入測試資料

In [151]:
test_df = pd.read_csv(測試資料路徑)
test_df.columns = df.columns
print(test_df.shape)
test_df.head()

(170, 27)


,T10,T50,T90,NA,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,98.6,119.2,146.1,40.133,1.142,0.616,0.217,3.745,2.577,4.028,0.557,7.669,5.9900,10.206,3.075,7.920,8.036,7.039,5.438,5.263,9.537,4.877,3.481,1.088,5.652,0.618,0.597
1,105.4,122.2,148.2,32.175,0.122,0.078,0.029,3.955,1.563,2.407,0.400,10.016,7.2633,6.573,2.809,9.544,9.815,5.135,5.482,6.793,10.850,4.308,3.963,1.069,6.025,0.585,0.484
2,105.8,122.7,149.2,31.428,0.098,0.064,0.020,3.920,1.447,2.296,0.396,10.236,7.3480,6.259,2.797,9.655,9.986,4.942,5.453,6.947,10.996,4.269,3.997,1.034,6.107,0.543,0.456
3,105.4,122.4,147.6,31.390,0.122,0.085,0.021,3.900,1.437,2.274,0.397,10.259,7.3560,6.234,2.820,9.670,10.146,4.928,5.476,6.953,11.026,4.264,3.982,1.013,6.054,0.545,0.449
4,105.4,122.6,148.2,32.418,0.072,0.049,0.015,3.873,1.369,2.206,0.489,10.213,7.3190,6.009,2.968,9.564,10.052,4.769,6.584,6.866,10.892,4.169,4.227,0.930,5.828,0.519,0.463


# 驗證/測試 資料 貼上所屬群Labels

In [152]:
from sklearn.manifold import Isomap
from sklearn.preprocessing import StandardScaler
import joblib

scaler = joblib.load('/content/drive/My Drive/台塑輕油案子/集成學習/tools/scaler_輕油資料使用.pkl')
iso = joblib.load('/content/drive/My Drive/台塑輕油案子/集成學習/tools/iso_輕油資料使用.pkl')
ART2 = joblib.load('/content/drive/My Drive/台塑輕油案子/集成學習/tools/ART2_輕油資料使用.pkl')

#scaler -> isomap -> ART2
val_df['cluster'] = ART2.transform(iso.transform(scaler.transform(val_df[x_columns])))
test_df['cluster'] = ART2.transform(iso.transform(scaler.transform(test_df[x_columns])))
print(val_df['cluster'].value_counts())
print(test_df['cluster'].value_counts())

3.0    154
0.0    122
1.0     89
2.0     48
4.0     40
Name: cluster, dtype: int64
3.0    77
0.0    58
1.0    30
2.0     5
Name: cluster, dtype: int64


# Lasso 特徵篩選器

In [153]:
#Modeling工具
from sklearn.base import TransformerMixin

class ModelTransformer(TransformerMixin):
  def __init__(self, model):
    self.model = model
  def set_params(self,alpha):
    self.model.set_params(alpha=alpha)
  def fit(self,X,y):
    self.model.fit(X,y)
    return self
  def transform(self,X):
    SelectColumn = (abs(self.model.coef_)!=0)
    return X[:,SelectColumn]

#Lasso特徵篩選器功能測試
from sklearn.linear_model import Lasso
import numpy as np

y_name = 'C5NP'

LassoTR = ModelTransformer(Lasso(0.5))
LassoTR.fit(np.array(test_df[x_columns]),np.array(test_df[y_name]))

print(np.array(test_df[x_columns]).shape)
print("==After Lasso==")
print(LassoTR.transform(np.array(test_df[x_columns])).shape)

(170, 4)
==After Lasso==
(170, 2)


# 交互項特徵產生器

In [154]:
#交互項特徵產生器
from sklearn.preprocessing import PolynomialFeatures

class featuresGenerator(TransformerMixin):
  def __init__(self):
    pass
  
  def fit(self,X,y):
    return self
  
  def transform(self,X):
    #交互項特徵
    interaction = PolynomialFeatures(2, interaction_only=True, include_bias=False)
    X_transformed = interaction.fit_transform(X) 
    return X_transformed

#交互項特徵產生器功能測試
print(np.array(test_df[x_columns]).shape)
f_tr = featuresGenerator()
print('==after features generator==')
print(f_tr.transform(np.array(test_df[x_columns])).shape)

(170, 4)
==after features generator==
(170, 10)


# 評分工具

In [0]:
from sklearn.metrics import r2_score
import numpy as np

def rmse(y_true, y_pred):
    return np.sqrt(((y_pred - y_true) ** 2).mean())

def mape(y_true, y_pred,dropinf=False):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    ape = np.abs((y_true - y_pred) / y_true)
    if dropinf == True:
      ape = ape[np.isfinite(ape)]
    return np.mean(ape) * 100

# 對驗證/測試 資料新增特徵工程欄位

In [0]:
def 特徵工程(df):
  df['T90-T50'] = df['T90']-df['T50']
  df['T90-T10'] = df['T90']-df['T10']
  df['T50-T10'] = df['T50']-df['T10']
  df['(90-50/90-10)*N+A']=(df['T90-T50']/df['T90-T10'])*df['NA']
  df['(50-10/90-10)*N+A']=(df['T50-T10']/df['T90-T10'])*df['NA']
  return df

In [157]:
import warnings 
warnings.simplefilter('ignore')
val_df = 特徵工程(val_df)
val_df.head()

,T10,T50,T90,NA,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A,cluster,T90-T50,T90-T10,T50-T10,(90-50/90-10)*N+A,(50-10/90-10)*N+A
2398,101.6,119.3,145.2,32.206,0.428,0.242,0.052,4.339,1.800,2.475,0.488,10.614,7.778,6.648,3.000,9.557,10.257,5.017,5.354,6.320,9.887,4.828,3.368,0.919,5.034,0.541,0.435,1.0,25.9,43.6,17.7,19.131546,13.074454
800,61.6,95.2,141.7,38.450,7.850,5.467,0.780,7.012,7.951,7.060,2.426,5.701,6.196,10.217,3.160,4.513,4.859,5.342,3.367,2.464,4.781,3.126,1.654,1.081,2.849,0.559,0.759,4.0,46.5,80.1,33.6,22.321161,16.128839
2288,104.7,120.1,144.5,32.807,0.233,0.175,0.025,3.026,0.786,2.314,0.389,10.812,7.907,7.313,3.213,9.748,10.480,5.221,5.787,6.614,10.756,4.250,3.384,0.735,4.688,0.585,0.326,1.0,24.4,39.8,15.4,20.112834,12.694166
2344,104.4,120.4,145.8,32.024,0.046,0.024,0.012,3.892,1.266,2.199,0.401,10.848,7.669,6.472,3.082,10.318,11.059,5.208,5.487,6.508,10.085,4.992,3.303,0.815,4.901,0.499,0.369,0.0,25.4,41.4,16.0,19.647575,12.376425
3615,92.4,112.1,142.5,51.471,2.306,1.492,0.277,2.732,2.560,4.559,0.609,5.690,5.916,18.572,3.592,5.128,6.858,9.576,4.610,2.546,7.436,5.706,2.365,0.747,4.039,0.823,0.782,3.0,30.4,50.1,19.7,31.231904,20.239096


In [158]:
test_df = 特徵工程(test_df)
test_df.head()

,T10,T50,T90,NA,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A,cluster,T90-T50,T90-T10,T50-T10,(90-50/90-10)*N+A,(50-10/90-10)*N+A
0,98.6,119.2,146.1,40.133,1.142,0.616,0.217,3.745,2.577,4.028,0.557,7.669,5.9900,10.206,3.075,7.920,8.036,7.039,5.438,5.263,9.537,4.877,3.481,1.088,5.652,0.618,0.597,0.0,26.9,47.5,20.6,22.727952,17.405048
1,105.4,122.2,148.2,32.175,0.122,0.078,0.029,3.955,1.563,2.407,0.400,10.016,7.2633,6.573,2.809,9.544,9.815,5.135,5.482,6.793,10.850,4.308,3.963,1.069,6.025,0.585,0.484,0.0,26.0,42.8,16.8,19.545561,12.629439
2,105.8,122.7,149.2,31.428,0.098,0.064,0.020,3.920,1.447,2.296,0.396,10.236,7.3480,6.259,2.797,9.655,9.986,4.942,5.453,6.947,10.996,4.269,3.997,1.034,6.107,0.543,0.456,0.0,26.5,43.4,16.9,19.189908,12.238092
3,105.4,122.4,147.6,31.390,0.122,0.085,0.021,3.900,1.437,2.274,0.397,10.259,7.3560,6.234,2.820,9.670,10.146,4.928,5.476,6.953,11.026,4.264,3.982,1.013,6.054,0.545,0.449,0.0,25.2,42.2,17.0,18.744739,12.645261
4,105.4,122.6,148.2,32.418,0.072,0.049,0.015,3.873,1.369,2.206,0.489,10.213,7.3190,6.009,2.968,9.564,10.052,4.769,6.584,6.866,10.892,4.169,4.227,0.930,5.828,0.519,0.463,0.0,25.6,42.8,17.2,19.390206,13.027794


# 定義x_columns,y_columns

In [159]:
x_columns = test_df.drop(y_columns+['cluster'],axis=1).columns.tolist()
print(x_columns)
print(y_columns)

['T10', 'T50', 'T90', 'NA', 'T90-T50', 'T90-T10', 'T50-T10', '(90-50/90-10)*N+A', '(50-10/90-10)*N+A']
['C5NP', 'C5IP', 'C5N', 'C6NP', 'C6IP', 'C6N', 'C6A', 'C7NP', 'C7IP', 'C7N', 'C7A', 'C8NP', 'C8IP', 'C8N', 'C8A', 'C9NP', 'C9IP', 'C9N', 'C9A', 'C10NP', 'C10IP', 'C10N', 'C10A']


# 調整y預測順序

In [160]:
new_y_columns = []
for i in y_columns:
  if i[-2:] == 'NP':
    new_i = i.replace('NP','IP')
  elif i[-2:] == 'IP':
    new_i = i.replace('IP','NP')
  else:
    new_i = i
  new_y_columns.append(new_i)
y_columns = new_y_columns
y_columns

['C5IP',
 'C5NP',
 'C5N',
 'C6IP',
 'C6NP',
 'C6N',
 'C6A',
 'C7IP',
 'C7NP',
 'C7N',
 'C7A',
 'C8IP',
 'C8NP',
 'C8N',
 'C8A',
 'C9IP',
 'C9NP',
 'C9N',
 'C9A',
 'C10IP',
 'C10NP',
 'C10N',
 'C10A']

# 定義預測流程

In [0]:
from tqdm import tqdm_notebook as tqdm

def Load_Modeling_predict(test_data,x_columns,y_columns,Sequential=False,model_dir_path=None):
  
  #初始化預測結果表
  predict_table = pd.DataFrame(np.zeros((len(test_data),len(y_columns))),index=test_data.index,columns=y_columns)
  #加入群label欄位
  predict_table['cluster'] = test_data['cluster']
  
  #對每一筆樣本作遍歷,判斷所屬的群
  for index in tqdm(test_data.index):
    one_row_data = test_data.loc[[index],x_columns]# one_row_data(一筆資料)
    cluster_name = int(test_data.loc[[index],['cluster']].values[0])#群代碼 int type
    #根據cluster_name和y_name呼叫model
    for y_name in y_columns:
      #呼叫Model
      dir_path = model_dir_path+'群代碼{}_'.format(cluster_name)
      file_name = '{}.pkl'.format(y_name)
      load_path = dir_path + file_name
      model = joblib.load(load_path)
      #模型預測&填入表格
      y_pred = model.predict(one_row_data)
      predict_table.loc[[index],[y_name]] = y_pred
      #把當前預測的y append到特徵(one_row_data)裡做為下一個y的特徵
      if Sequential == True:
        one_row_data[y_name] = y_pred
  #輸出表格
  return predict_table

# call先前保存好的Model對val_df做預測

In [0]:
rf_pred_val = Load_Modeling_predict(val_df,
                                    list(x_columns),
                                    list(y_columns),
                                    Sequential=True,
                                    model_dir_path = rf_model_dir_path)

In [0]:
knn_pred_val = Load_Modeling_predict(val_df,
                                     list(x_columns),
                                     list(y_columns),
                                     Sequential=True,
                                     model_dir_path = knn_model_dir_path)

In [0]:
svm_pred_val = Load_Modeling_predict(val_df,
                                     list(x_columns),
                                     list(y_columns),
                                     Sequential=True,
                                     model_dir_path = svm_model_dir_path)

# call先前保存好的Model對test_df做預測

In [0]:
rf_pred_test = Load_Modeling_predict(test_df,
                                    list(x_columns),
                                    list(y_columns),
                                    Sequential=True,
                                    model_dir_path = rf_model_dir_path)

In [0]:
knn_pred_test = Load_Modeling_predict(test_df,
                                    list(x_columns),
                                    list(y_columns),
                                    Sequential=True,
                                    model_dir_path = knn_model_dir_path)

In [0]:
svm_pred_test = Load_Modeling_predict(test_df,
                                    list(x_columns),
                                    list(y_columns),
                                    Sequential=True,
                                    model_dir_path = svm_model_dir_path)

# Second level learning model via XGBoost

In [194]:
import xgboost as xgb

table = pd.DataFrame()

for y_name in y_columns:
  #3個model在val上的預測結果
  predict_val = pd.DataFrame({'RF':rf_pred_val[y_name],
                            'SVM':svm_pred_val[y_name],
                            'KNN':knn_pred_val[y_name],
                            'True':val_df[y_name]},
                           index=val_df.index)
  #三個model在test上的預測結果
  predict_test = pd.DataFrame({'RF':rf_pred_test[y_name],
                            'SVM':svm_pred_test[y_name],
                            'KNN':knn_pred_test[y_name],
                            'True':test_df[y_name]},
                           index=test_df.index)
  
  #用val做訓練
  x_train = predict_val.drop('True',axis=1)
  y_train = predict_val['True']
  #用test做預測
  x_test = predict_test.drop('True',axis=1)
  y_test = predict_test['True']
  
  #xgboost做level2 model
  gbm = xgb.XGBRegressor()
  
  #訓練
  gbm.fit(x_train, y_train)
  
  #預測
  predict = gbm.predict(x_test)
  
  #加入評估結果至table
  table = table.append(evaluate(y_test,predict,y_name=y_name))

[07:56:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:56:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:56:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:56:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:56:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:56:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:56:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[07:56:00] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [195]:
table.loc['AVG'] = table.mean(axis=0)
table

,R2,RMSE,MAPE,MAPE修正
C5IP,0.568944,0.335972,57.570325,57.570325
C5NP,0.729946,0.444099,42.671820,42.671820
C5N,0.264554,0.117400,35.985831,35.985831
C6IP,0.420316,0.795830,14.560172,14.560172
C6NP,-0.258952,0.453948,8.641208,8.641208
C6N,0.662279,0.962387,9.463475,9.463475
C6A,-1.985780,0.295218,25.145906,25.145906
C7IP,0.814841,0.475485,5.781882,5.781882
C7NP,0.946455,0.508907,4.629941,4.629941
C7N,0.947021,0.959209,5.242091,5.242091
